In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="h-e-l-l-o/email-spam-classification-merged")

c:\Users\USER\Documents\CC\Foundatuin ML\projects\CSCN8010\venv\pytorch_cpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

df = pd.read_csv("./csv/mail.csv")

In [3]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
len(df)

1632

In [5]:
!pip install nltk

In [6]:
import string
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
## TODO: Clean the sentences

def nlp_preprocess(sentence):
        
    # Tokenization
    wt_t = word_tokenize(sentence)

    # Stop-Words Removal
    stopwords.words("english")
    st_t = [word for word in wt_t if word not in stopwords.words("english")]

    # Remove punctuation
    punc_list = list(string.punctuation)
    punc_list.append('`')
    punc_list.append('--')
    pr_t = [word for word in st_t if word not in punc_list]

    # Lower all words
    l_t = [word.lower() for word in pr_t]

    # Lemmatization
    # The reason I chose the Lemmatizer it's because when every time I use both function to transfer the word, Lemmatizer could always return a text that is easier to recognize.
    lemmatizer = WordNetLemmatizer()
    lemm_t = [lemmatizer.lemmatize(word) for word in l_t]
    
    filter_len_1 = [item for item in lemm_t if len(item) != 1]

    # POS Tagging
    # pos_t = nltk.pos_tag(word_tokenize(" ".join(lemm_t)))

    # This function should return list of tokens after applying NLP Tasks explained above
    return filter_len_1

In [8]:
import math

results = {"prob": [], "label": [], "new_content": []}
gap = 500

for index, row in df.iterrows():
    # print(len(row['Content']))
    new_c = " ".join(nlp_preprocess(row["Content"]))
    results["new_content"].append(new_c)

    le = len(new_c)

    if len(new_c) < gap:
        result = pipe(new_c)
        results["prob"].append(result[0]["score"])
        results["label"].append(result[0]["label"])
    else:
        times = math.ceil(le / gap)

        spam = []
        ham = []

        for i in range(times):
            b = gap * i
            e = gap * i + 1
            r = pipe(new_c[b:e])

            if r[0]["label"] == "Ham":
                ham.append(r[0]["score"])
            else:
                spam.append(r[0]["score"])

        sum_spam = sum(spam)
        sum_ham = sum(ham)

        results["label"].append("Spam" if sum_spam > sum_ham else "Ham")
        results["prob"].append(abs(sum_ham - sum_spam) / times)

In [9]:
df['label'] = results["label"]
df['score'] = results["prob"]

In [10]:
df['label'].value_counts()

label
Spam    1210
Ham      422
Name: count, dtype: int64

In [11]:
df.to_csv('./csv/mail_labeling.csv')